# Patiently -- Initial Algorithm for Patient Matching Problem 

We followed Office Ally's POC algorithm closely with a Python/SQLite database implementation. In addition to using hash tokens and Soundex tokens, we also incorporated street address features in the matching and worked on improving efficiency.

In [21]:
import pandas as pd
import hashlib
from pyphonetics import Soundex, RefinedSoundex

In [2]:
# Load data from input CSV (also loaded in db)
data = pd.read_csv("Patient_Matching_Data.csv")
data = data.loc[:, :"Current Zip Code"]
data.head()

## Standardization of Fields
We standardize data by using all upper case letters and retaining only alphanumerics.

In [4]:
# Missing values are replaced with empty string or 0s
data['First Name'] = data['First Name'].fillna("").str.upper().str.replace('\W', '')
data['MI'] = data['MI'].fillna("").str.upper().str.replace('\W', '')
data['Last Name'] = data['Last Name'].fillna("").str.upper().str.replace('\W', '')
data['Current Street 1'] = data['Current Street 1'].fillna("").str.upper().str.replace('\W', '')
data['Current Street 2'] = data['Current Street 2'].fillna("").str.upper().str.replace('\W', '')

# For sex, only keep first character and represent missing Na as "U"
data['Sex'] = data['Sex'].str.upper().str[0]
data['Sex'] = data['Sex'].fillna("U")

# Convert zip code to alphanumeric string or 0
data['Current Zip Code'] = data['Current Zip Code'].fillna(0).astype(int).astype(str).str[:5]
data['Current City'] = data['Current City'].fillna("").str.upper().str.replace('\W', '')
data.head()

,GroupID,PatientID,Patient Acct #,First Name,MI,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,Current City,Current State,Current Zip Code
0,1,1,247028705-7,SUTTON,J,POWER,9/20/1945,M,1858SULLIVANPARKWAY,,FRESNO,California,93726
1,1,2,NaN,SUTTIN,JAMES,POWER,9/21/1945,M,1859SULLIVANPARKWAY,2,FRESNO,California,93726
2,1,3,247028705-7,SUTTON,J,POWER,9/20/1945,M,1858SULLIVANPARKWAY,,FRESNO,CA,93726
3,1,4,NaN,SUTTON,,POWER,9/20/1954,M,1858SULLIVANPARKWAY,,FRESNO,California,93726
4,1,5,NaN,SUTTON,,POWER,9/20/1954,M,1858SULLIVANPKWAY,APT2,FRESNO,California,93726


In [5]:
# Normalize the states by turning abbreviations into text
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

data['Current State'] = data['Current State'].replace(to_replace=states, value=None)
data['Current State'] = data['Current State'].str.upper().str.replace('\W', '')

In [6]:
# Standardize date of births, and fill missing values with 0s
data['Date of Birth'] = pd.to_datetime(data['Date of Birth'], errors='coerce').dt.date
data['Date of Birth'] = data['Date of Birth'].fillna(0).astype(str)
data

,GroupID,PatientID,Patient Acct #,First Name,MI,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,Current City,Current State,Current Zip Code
0,1,1,247028705-7,SUTTON,J,POWER,1945-09-20,M,1858SULLIVANPARKWAY,,FRESNO,CALIFORNIA,93726
1,1,2,NaN,SUTTIN,JAMES,POWER,1945-09-21,M,1859SULLIVANPARKWAY,2,FRESNO,CALIFORNIA,93726
2,1,3,247028705-7,SUTTON,J,POWER,1945-09-20,M,1858SULLIVANPARKWAY,,FRESNO,CALIFORNIA,93726
3,1,4,NaN,SUTTON,,POWER,1954-09-20,M,1858SULLIVANPARKWAY,,FRESNO,CALIFORNIA,93726
4,1,5,NaN,SUTTON,,POWER,1954-09-20,M,1858SULLIVANPKWAY,APT2,FRESNO,CALIFORNIA,93726
5,2,6,NaN,SUTTON,,POWERS,2000-12-15,F,1619MAPLELANE,,FRESNO,CALIFORNIA,93726
6,2,7,NaN,SUTTON,,POWERS,2000-12-15,F,1619MAPLELANE,,FRESNO,CALIFORNIA,93726
7,3,8,757393348-7,MIKEL,L,TRAYTON,2000-09-20,M,715JANAPOINT,,APACHEJUNCTION,ARIZONA,85219
8,3,9,757393348-7,MIKEL,LEE,TRAYTON,2000-09-20,M,715JANAPOINT,,APACHEJUNCTION,ARIZONA,85219
9,3,10,757393348-7,MICHAEL,,TRAYTON,2000-09-20,M,715JANAPOINT,,APACHEJUNCTION,ARIZONA,85219


## Partial Hashing to Compute Distinct Patients
For accurate records, the hash of partial names, birthdate and sex for distinct patients should give a good initial grouping of records. Assuming a low collision rate, we can assume that the different hashes represent different patients.

In [7]:
# Compute partial SHA-1 hashes using birthdate, partial names, sex
salt = "LAHACKS"

def compute_hash(fields):
    res = []
    for first, last, date_of_birth, sex in fields:
        h = hashlib.sha1()
        # Partial hashing only uses first 3 chars of names
        partial_first = (first + ("_" * 3))[:3]
        partial_last = (last + ("_" * 3))[:3]
        date_of_birth = (date_of_birth + ("_" * 10))[:10]
        h.update((salt + date_of_birth + "~" + sex + "~" + partial_first + "~" + partial_last + "XXX").encode('utf-8'))
        res.append(h.hexdigest())
    return res

In [8]:
data['parHash'] = compute_hash(list(zip(data["First Name"], data["Last Name"], data["Date of Birth"], data["Sex"])))

In [9]:
# Deduplicate by the partial hash, so all accurate records are grouped
dedupped = data.groupby(["parHash"])
dedup_by_hash = data.groupby(["parHash"])["PatientID"].apply(list).reset_index(name='patient_ids') 

In [10]:
# Build reference table for dedupped groups and important information
reference = dedupped.first().reset_index()\
[["parHash", "First Name", "Last Name", "Date of Birth", "Sex", "Current Street 1", "Current City", "Current State", "Current Zip Code"]]

In [11]:
reference = reference.merge(dedup_by_hash)
reference.head()

,parHash,First Name,Last Name,Date of Birth,Sex,Current Street 1,Current City,Current State,Current Zip Code,patient_ids
0,01a4ca012df8139ceea9c1adbfb4970f7bc14892,MIKE,TRAYTON,2000-09-02,M,715JANAPOINT,APACHEJUNCTION,ARIZONA,85219,[11]
1,02e52b2bee3e42c3559731154a3d28a66a277e8d,MAYNARD,SKIRVAN,2007-04-24,M,485HANSONSTREETE,PHOENIX,ARIZONA,85010,"[31, 33]"
2,0838ea2d08a5715c8e34e0d47832c1b97eecf5db,KATHARINA,STOPPER,1960-03-19,F,61256MERRICKPLACE,ALBUQUERQUE,NEWMEXICO,87190,"[70, 72, 73, 74]"
3,0a472ec641edaba53fa8433463e3a32218f104a4,ANNE,KEIG,1935-08-15,F,474ANNAMARKAVE,CONROE,TEXAS,77305,[170]
4,0ced8af7010e01cac2286831c7b4d06db541822b,BYROM,VANINI,1985-11-08,M,9BELLGROVESTREET,SILVERSPRING,MARYLAND,20904,"[95, 98, 99]"


## Pairwise De-duplication with Similarity Heuristics 

Our heuristics use Soundex tokens and Levenshtein distances to estimate whether two patient groups are similar enough to be in the same patient group. The similarity estimations also considers whether birthdates, gender, and street addresses match.

In [12]:
rs = RefinedSoundex()
s = Soundex()

THRESHOLD = 1.0
LEV_THRESHOLD = 3
def is_similar(row1, row2):
    similarity = 0
    # Heuristics for determining similarity
    if row1["First Name"] and row2["First Name"]:
        if s.sounds_like(row1["First Name"], row2["First Name"]) or \
        rs.distance(row1["First Name"], row2["First Name"], metric='levenshtein') < LEV_THRESHOLD:
            similarity += 0.4
    if row1["Last Name"] and row2["Last Name"]:
        if s.sounds_like(row1["Last Name"], row2["Last Name"]) or \
        rs.distance(row1["Last Name"], row2["Last Name"], metric='levenshtein') < LEV_THRESHOLD:
            similarity += 0.4
        
    if row1["Date of Birth"] and row2["Date of Birth"] and row1["Date of Birth"] != row2["Date of Birth"]:
        similarity -= 0.1
    else:
        similarity += 0.4
        
    if (row1["Sex"] == "M" and row2["Sex"] == "M") or (row1["Sex"] == "F" and row2["Sex"] == "F"):
        similarity += 0.2
    elif (row1["Sex"] == "M" and row2["Sex"] == "F") or (row1["Sex"] == "F" and row2["Sex"] == "M"):
        similarity -= 0.4
        
    if row1["Current Street 1"] and row2["Current Street 1"] and \
    row1["Current Street 1"] == row2["Current Street 1"]:
        similarity += 0.4
    
    if row1["Current Zip Code"] and row2["Current Zip Code"] and \
    row1["Current Zip Code"] == row2["Current Zip Code"]:
        similarity += 0.2
        
    if row1["Current City"] and row2["Current City"] and \
    row1["Current City"] == row2["Current City"]:
        similarity += 0.2
    return similarity > THRESHOLD

## Disjoint-set Data Structure for Efficient Grouping

This data structure efficiently maintains the sets of patient groups as similarities are discovered.

In [13]:
class DSU:
    """Disjoint-set data structure for maintaining sets of patient records.
    """
    def __init__(self, n):
        self.fa = [x for x in range(n)]

    def find(self, x):
        if x == self.fa[x]:
            return x
        self.fa[x] = self.find(self.fa[x])
        return self.fa[x]

    def union(self, x, y):
        a = self.find(x)
        b = self.find(y)
        self.fa[b] = a

In [14]:
dsu = DSU(len(reference))

for index1, row1 in reference.iterrows():
    for index2, row2 in reference.iterrows():
        if index1 <= index2:
            break
        else:
            if is_similar(row1, row2):
                dsu.union(index1, index2)

In [15]:
group_dict = {}
# Loop through the previous group ids and determine their final groupings
for group_id in range(len(reference)):
    if dsu.find(group_id) in group_dict:
        group_dict[dsu.find(group_id)].append(group_id)
    else:
        group_dict[dsu.find(group_id)] = [group_id]

# Loop through the final groupings to get the patients
counter = 0
patients_dict = {}
for final_id, group_ids in group_dict.items():
    patients_dict[counter] = []
    for group_id in group_ids:
        patient_ids = reference.iloc[[group_id]]["patient_ids"].tolist()
        patients_dict[counter] += patient_ids[0]
    counter += 1

In [16]:
patient_to_group = {}
for i, patients in patients_dict.items():
    for p in patients:
        patient_to_group[p] = i

In [19]:
original = pd.read_csv("Patient_Matching_Data.csv")

original["Temp Group ID"] = original["PatientID"].replace(to_replace=patient_to_group)

groupings_list = original["Temp Group ID"].tolist()
# Renumber whenever id changes so that it is ordered by patient number
renumbering = {}
counter = 0
final_groupings = []
for i in groupings_list:
    if i not in renumbering:
        counter += 1
        renumbering[i] = counter
    final_groupings.append(renumbering[i])

## Output Patient-Final Group ID Mappings

Having completed patient-matching, we can return the final group IDs as a CSV or as a new table in the database. 

In [20]:
original.insert(0, 'Final Group ID', final_groupings)
original

,Final Group ID,GroupID,PatientID,Patient Acct #,First Name,MI,Last Name,Date of Birth,Sex,Current Street 1,...,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code,Temp Group ID
0,1,1,1,247028705-7,Sutton,J,Power,9/20/1945,Male,1858 Sullivan Parkway,...,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,New York,10557.0,31
1,1,1,2,NaN,Suttin,James,Power,9/21/1945,Male,1859 Sullivan Parkway,...,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,New York,10557.0,31
2,1,1,3,247028705-7,Sutton,J,Power,9/20/1945,Male,1858 Sullivan Parkway,...,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31
3,1,1,4,NaN,Sutton,NaN,Power,9/20/1954,Male,1858 Sullivan Parkway,...,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31
4,1,1,5,NaN,SUTTON,NaN,POWER,9/20/1954,Male,1858 SULLIVAN PKWAY,...,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31
5,2,2,6,NaN,Sutton,NaN,Powers,12/15/2000,Female,1619 Maple Lane,...,93726.0,Sutton,NaN,Lore,NaN,NaN,NaN,NaN,NaN,35
6,2,2,7,NaN,Sutton,NaN,Powers,12/15/2000,Female,1619 Maple Lane,...,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35
7,3,3,8,757393348-7,Mikel,L,Trayton,9/20/2000,Male,715 Jana Point,...,85219.0,NaN,NaN,NaN,81089 Kenwood Park,NaN,Chattanooga,Tennessee,37416.0,0
8,3,3,9,757393348-7,Mikel,Lee,Trayton,9/20/2000,Male,715 Jana Point,...,85219.0,NaN,NaN,NaN,81089 Kenwood Park,NaN,Chattanooga,Tennessee,37416.0,0
9,3,3,10,757393348-7,Michael,NaN,Trayton,9/20/2000,Male,715 Jana Point,...,85219.0,NaN,NaN,NaN,81089 Kenwood Park,7,Chattanooga,Tennessee,37416.0,0
